<h1>NFW internal Halo Profiles</h1>
This notebook contains code to output the halo profiles of the given mass and redshift at the given scale in fourier space.

In [59]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

<h2>Setting up WMAP1 Cosmology parameters below</h2>
Source: <a href='https://arxiv.org/pdf/astro-ph/0302209.pdf'>arXiv paper</a> on <i>First Year Wilkinson Microwave Anisotropy Probe (WMAP) Observations:Determination of Cosmological Parameters</i> by Spergel et. al.

In [51]:
z=0.0   # Redshift
del_crit=1.69/(1.0+z)   # Critical overdensity for spherical collapse at given redshift (from linear theory) for EdS Universe
## for a more detailed description of calculating del_crit and its evolution with redshift for a LCDM Universe, check arXiv:0011295

#'''
#WMAP1 Cosmology (Tinker uses this for MF paper)
omega_m=0.270062   # Density parameter for matter in the Universe WMAP1 Cosmology (Niladri's=0.276)
h=0.72  # 100h km /s MPc is Hubble constant today WMAP1 Cosmology (Niladri's=0.7)
#'''

'''
#Niladri's cosmology
omega_m=0.276
h=0.7
'''

M_sun=1.989e+30  # M_sun in kg
Mega_parsec=3.086e+22  # Mega parsec in metres

rho_cr=((3*np.power((100*h),2))/(8*np.pi*6.673e-11))*(Mega_parsec*1e+6*h/(M_sun))
# critical density of the Universe today 3H^2/8Pi*G in units M_sun.h^-1/MPc^3
rho_m=omega_m*rho_cr*((1+z)**3)  # where a=1/(1+z)

eps=1e-5   # To avoid divide by 0 and to set a cutoff limit for negligibly low values
finesse=1000   # Defines the granularity in calculating Si and Ci integrals

In [52]:
# This cell holds code to compute the variance for a given smoothening scale R defined by the mass passed as argument
# make sure the k's are log spaced in the source file for Power Spectrum
PS=np.loadtxt("./../Normalize PS/NormalizedDimensionlessPS.txt")
k=PS[:,0]
ps=PS[:,1]  # ps=k^3*P/2*pi^2

const1=np.cbrt(3.0/(4*np.pi*rho_m))

# Smoothing window functions Fourier Transformed in k-space
def TopHat(k,R):
    return (3/np.power(k*R,3))*(np.sin(k*R)-(k*R)*np.cos(k*R))

def variance(m):
    R=const1*np.cbrt(m)
    function=ps*(TopHat(k,R)**2)
    result=np.trapz(function,np.log(k))
    return result

In [53]:
variance(1e+10)-del_crit**2

14.401714352049536

In [54]:
# M_star(z) is the characteristic mass scale at which Nu(m,z)=del_crit**2/variance(Mstar)**2=1
# Goal is to find the mass at which variance(sigma^2) is equal to del_crit^2
def Mstar():
    Mlower=1e+10
    Mupper=1e+16
    Mst=-1
    
    limit=1e+10   # Upper threshold on number of iterations. If answer not found till then, the loop breaks
    lim_cnt=0
    
    while True:
        Mcheck=(Mupper+Mlower)/2.0
        x=variance(Mcheck)-(del_crit**2)
        if(0.0<=x<=eps):
            Mst=Mcheck
            break
        if(x<0.0):
            Mupper=Mcheck
        if(x>eps):
            Mlower=Mcheck
        
        if lim_cnt<=limit:
            lim_cnt+=1
        else:
            print("Result not found by this method after "+str(limit)+" number of iterations")
            exit()
            break
            
    return Mst

Mstar()

3477086960785.091

In [79]:
# Return the density at given scale k and mass of halo m for spherically symmetric profile truncated at virial radius
# reference page 34 in Cooray & Sheth

# Robust wrt to this
f1=lambda x: np.sin(x)/x
def Si(t):
    result=integrate.quad(f1,0,t)
    return result[0]

# Possible fault regarding consideration of values of x
########## Overly sensitive on the values of t
f2=lambda x: -1.0*np.cos(x)/x
def Ci(t):
    result=integrate.quad(f2,t,np.inf)
#    return result[0]
    return 1.0

# Returns the scale radius defined as Rs=R200/c ; where R200 is found from total mass of Halo and background density 
def Rs(M,c):
    rho_200=200.0*rho_m
    r200=np.cbrt(M*3.0/(4.0*np.pi*rho_200))
    return r200/c

# Returns rho_s given a concentration mass relation (defined in Cooray Sheth Page 32 bottom most for NFW & M99)
def Rhos(M,r_s,c):
    term1=4.0*np.pi*(r_s)**3
    term2=np.log(1.0+c)-c/(1.0+c)
    return M/(term1*term2)
    
# Returns NFW profile in K space
m_st=Mstar()
def u_NFW(k,m):
    c=(9.0/(1.0+z))*((m/m_st)**-0.13)
    r_s=Rs(m,c)
    rho_s=Rhos(m,r_s,c)
    
    util1=k*r_s
    util2=(1+c)*util1
    
    term1=4.0*np.pi*rho_s*r_s/m
    term2=np.sin(util1)*(Si(util2)-Si(util1))
    term3=-1.0*np.sin(c*util1)/util2
    term4=np.cos(util1)*(Ci(util2)-Ci(util1))
    print(util1)
    print(util2)
    
    result=term1*(term2+term3+term4)
    return result
u_NFW(1.0,1.0e+13)

0.09596196695112207
0.848796743006126


-61.863274466532154

In [68]:
f2=lambda x: -1.0*np.cos(x)/x
def Ci(t):
    result=integrate.quad(f2,t,np.inf)
    return result[0]
Ci(0.16)

-4.264337504458664